In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0`

import $ivy.`org.apache.kafka:kafka-clients:3.5.1`
import $ivy.`org.testcontainers:testcontainers:1.19.3`
import $ivy.`org.testcontainers:kafka:1.19.3`

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$

In [2]:

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._

import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val kafkaContainer = new KafkaContainer(
  DockerImageName.parse("confluentinc/cp-kafka:7.6.5")
)

kafkaContainer.start()

val bootstrapServers = kafkaContainer.getBootstrapServers
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

def readFromKafkaSpark(topic: String): DataFrame = {
  spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", topic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")
}

val kafkaTopic = "my-topic-scala"


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 16:43:21 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 16:43:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 16:43:21 INFO SparkContext: Running Spark version 3.5.0
25/03/30 16:43:21 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 16:43:21 INFO SparkContext: Java version 17.0.14
25/03/30 16:43:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 16:43:21 INFO ResourceUtils: ==============================================================
25/03/30 16:43:21 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 16:43:21 INFO ResourceUtils: ==============================================================
25/03/30 16:43:21 INFO SparkCon

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._
import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties
spark: SparkSession = org.apache.spark.sql.SparkSession@1458a8eb
import spark.implicits._
kafkaContainer: KafkaContainer = GenericContainer(extraHosts=[], image=RemoteDockerImage(imageName=confluentinc/cp-kafka:7.6.5, imagePullPolicy=DefaultPullPolicy(), imageNameSubstitutor=org.testcontainers.utility.ImageNameSubstitutor$LogWrappedImageNameSubstitutor@444205fa), volumesFroms=[], linkedContainers={}, startupCheckStrategy=org.testcontainers.containers.startupcheck.IsRunningStartupCheckStrategy@25a9c8f7, startupAttempts=1, workingDirectory=null, shmSize=null, copyToFileContainerPathMap={}, copyToTransferableContainerPathMap={}, dependencies=[], dockerClient=LazyDockerClient, containerId=24f996cd470cae96c11d3f4a339896af0a352d570c2693216a9f838337d92223, containerInfo

In [3]:
val data = Seq("hi", "there", "from", "scala")

sendToKafka(kafkaTopic, data)

25/03/30 16:43:25 INFO Metadata: [Producer clientId=producer-1] Cluster ID: V9utvC3-RQqWJ-T04Ee3ng
25/03/30 16:43:26 INFO TransactionManager: [Producer clientId=producer-1] ProducerId set to 0 with epoch 0
25/03/30 16:43:26 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 5 : {my-topic-scala=LEADER_NOT_AVAILABLE}
25/03/30 16:43:26 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 6 : {my-topic-scala=LEADER_NOT_AVAILABLE}
25/03/30 16:43:26 INFO Metadata: [Producer clientId=producer-1] Resetting the last seen epoch of partition my-topic-scala-0 to 0 since the associated topicId changed from null to QAKOnV4MTn-Mm1Gb5o5JjQ


data: Seq[String] = List("hi", "there", "from", "scala")

In [4]:
readFromKafkaSpark(kafkaTopic).show()

25/03/30 16:43:26 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 16:43:26 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.
25/03/30 16:43:27 INFO CodeGenerator: Code generated in 175.413951 ms
25/03/30 16:43:27 INFO CodeGenerator: Code generated in 22.051742 ms
25/03/30 16:43:28 INFO AdminClientConfig: AdminClientConfig values: 
	auto.include.jmx.reporter = true
	bootstrap.servers = [PLAINTEXT://localhost:57759]
	client.dns.lookup = use_all_dns_ips
	client.id = 
	connections.max.idle.ms = 300000
	default.api.timeout.ms = 60000
	metadata.max.age.ms = 300000
	metric.reporters = []
	metrics.num.samples = 2
	metrics.recording.level = INFO
	metrics.sample.window.ms = 30000
	receive.buffer.bytes = 65536
	reconnect.backoff.max.ms = 1000
	reconnect.backoff.ms = 50
	request.timeout.ms = 30000
	retries = 2147483647
	retry.backoff.ms = 100
	sasl.client.callback.handler.class = null
	sasl.jaas.c

+-------+
|message|
+-------+
|     hi|
|  there|
|   from|
|  scala|
+-------+

